# Ensemble

***

### 작업 제 2유형 SCORE 달성에 가장 안전한 방법

#### 앙상블이란?
- 집단지성과 같은 의미
- 여러 개 예측값을 결합하자!
- 산술 평균, 기하 평균 등 다양하다.
- 독립적인 예측값 결합으로 과적합 방지 가능
***

### KFold와 StratifiedKFold 사용!
KFold => 랜덤 분포 랑 StratifiedKFold=> 균등 분포
### 프로세스
- for loop문 구성
- 학습 및 검증 데이터 분할
- 각 FOLD별로 모델이 데이터를 학습하고 test 데이터에 대해 예측 값 생성
- K개의 FOLD 예측 값을 앙상블한다.

예를 들어 10FOLD로 학습 및 검증을 한다.

- 학습 : 9개 FOLD로 학습을 한다.
- 검증 : 1개 FOLD로 검증을 한다.

모델 학습을 총 10번한다. -> 모델이 총 10개라는 의미!!

알고리즘은 같지만 모델이 사용하는 독립변수, 종속변수 분포는 다르다.

10개 각 모델로 test 데이터를 각각 예측한다. -> 10개의 예측값 생성

10개의 예측값에 대해서 산술평균을 활용하자!

In [1]:
import pandas as pd

In [13]:
X = pd.read_csv('X_train.csv', encoding = 'cp949')
y = pd.read_csv("y_train.csv", encoding = 'cp949').gender
test = pd.read_csv('X_test.csv', encoding = 'cp949')

In [14]:
X

,cust_id,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,0,68282840,11264000,6860000.0,기타,강남점,19,3.894737,0.527027,17
1,1,2136000,2136000,300000.0,스포츠,잠실점,2,1.500000,0.000000,1
2,2,3197000,1639000,NaN,남성 캐주얼,관악점,2,2.000000,0.000000,1
3,3,16077620,4935000,NaN,기타,광주점,18,2.444444,0.318182,16
4,4,29050000,24000000,NaN,보석,본 점,2,1.500000,0.000000,85
...,...,...,...,...,...,...,...,...,...,...
3495,3495,3175200,3042900,NaN,골프,본 점,1,2.000000,1.000000,0
3496,3496,29628600,7200000,6049600.0,시티웨어,부산본점,8,1.625000,0.461538,40
3497,3497,75000,75000,NaN,주방용품,창원점,1,1.000000,0.000000,0
3498,3498,1875000,1000000,NaN,화장품,본 점,2,1.000000,0.000000,39


In [15]:
y

0       0
1       0
2       1
3       1
4       0
       ..
3495    1
3496    1
3497    0
3498    0
3499    0
Name: gender, Length: 3500, dtype: int64

In [16]:
X.isnull().sum()

cust_id       0
총구매액          0
최대구매액         0
환불금액       2295
주구매상품         0
주구매지점         0
내점일수          0
내점당구매건수       0
주말방문비율        0
구매주기          0
dtype: int64

In [23]:
X['주구매상품'].value_counts() # 3000개의 데이터양 치고 카테고리가 너무 많아요


기타        595
가공식품      546
농산물       339
화장품       264
시티웨어      213
디자이너      193
수산품       153
캐주얼       101
명품        100
섬유잡화       98
골프         82
스포츠        69
일용잡화       64
모피/피혁      57
육류         57
남성 캐주얼     55
구두         54
건강식품       47
차/커피       44
피혁잡화       40
아동         40
축산가공       35
주방용품       32
셔츠         30
젓갈/반찬      29
주방가전       26
트래디셔널      23
남성정장       22
생활잡화       15
주류         14
가구         10
커리어         9
대형가전        8
란제리/내의      8
식기          7
액세서리        5
침구/수예       4
통신/컴퓨터      3
보석          3
남성 트랜디      2
소형가전        2
악기          2
Name: 주구매상품, dtype: int64

In [24]:
X.fillna(0, inplace = True) # 결측치를 0으로 채운다. 

In [25]:
test.fillna(0, inplace = True) # test도 채워줍니다.

In [26]:
X.drop(['cust_id', '주구매상품', '주구매지점'], axis = 1, inplace = True) # drop할 것들 삭제

In [27]:
test.drop(['cust_id', '주구매상품', '주구매지점'], axis = 1, inplace = True)

In [28]:
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import numpy as np
# 어디 라이브러리에 model이 있는지 외우자!

In [29]:
skf = StratifiedKFold(n_splits = 10, random_state = 42, shuffle = True) # 모델을 불러오기

10fold로 나누었기 때문에 각 fold에는 데이터가 350개씩 들어간다.

In [30]:
rf_pred = np.zeros((test.shape[0])) # 초기화 리스트를 만들어주기 
# zeros를 ones로 하면 1로 채워진다.

In [31]:
rf_pred

array([0., 0., 0., ..., 0., 0., 0.])

예측값으로 치환하기 위해 틀을 만들어둔다.

In [32]:
len(rf_pred) # test 개수 

2482

In [33]:
test.shape[0]

2482

In [37]:
### tr_idx, val_idx는 학습데이터에 대해 각각 학습용, 검증용을 의미한다.
### 10fold로 지정해두었기 때문에 학습용 : 3150개, 검증용 : 350개
for tr_idx, val_idx in skf.split(X, y) :
    #print(f'학습용 : {len(tr_idx)} / 검증용 : {len(val_idx)}')
    print(tr_idx[:10], val_idx[:10], '\n') # 각 데이터의 인덱스 앞에서 10개 출력
    print(len(tr_idx), len(val_idx))

[0 1 2 3 4 5 6 7 8 9] [16 17 26 34 38 46 57 66 67 69] 

3150 350
[ 2  3  4  5  6  7  8  9 10 11] [ 0  1 25 28 30 50 53 83 89 92] 

3150 350
[ 0  1  2  3  7  8  9 11 13 14] [ 4  5  6 10 12 41 45 47 55 62] 

3150 350
[ 0  1  3  4  5  6  7  8  9 10] [  2  40  65  90  96  97  99 120 144 151] 

3150 350
[0 1 2 3 4 5 6 7 8 9] [ 27  44  48  54  64  72  75  77  85 102] 

3150 350
[ 0  1  2  3  4  5  6  7  9 10] [ 8 11 20 21 33 60 63 73 81 86] 

3150 350
[ 0  1  2  4  5  6  7  8 10 11] [ 3  9 19 23 32 43 58 74 78 79] 

3150 350
[0 1 2 3 4 5 6 7 8 9] [15 35 36 39 51 59 68 80 82 95] 

3150 350
[0 1 2 3 4 5 6 7 8 9] [14 22 24 31 42 49 52 76 87 91] 

3150 350
[ 0  1  2  3  4  5  6  8  9 10] [  7  13  18  29  37  56  61  71  84 105] 

3150 350


모델이 사용하는 데이터가 각각 다르므로 독립적인 모델이 10개로 구성되었다.

In [38]:
from sklearn.metrics import accuracy_score

In [45]:
rf_pred = np.zeros((test.shape[0]))

# pred y 값 초기화 리스트 생성
for tr_idx, val_idx in skf.split(X, y) :

    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]    
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    # 10Kfold로 test랑 val 데이터 나눠주기 

    model = RandomForestClassifier(random_state = 42) # 랜포 만들어주기
    model.fit(tr_x, tr_y) # 학습시키기
    
    
    val_pred = model.predict_proba(val_x)[:, 1] # proba 예측하기 
    
    
    ###
    val_pred = val_pred >= 0.5 # 0.5 이상인 확률 값은 1로 치환 그렇지 않은 확률은 0으로 치환 
    
    val_pred = val_pred.astype(int)
    ###
    
    val_score = accuracy_score(val_y, val_pred) # 정확도 뽑기 예측값맞은 확률 
    print(val_score) # 프린트 
    fold_pred = model.predict_proba(test)[:, 1] / 10 # 테스트 케이스를 넣어서 예측값 뽑기 
    rf_pred += fold_pred # 평균 정확도 구하기 

# rf_pred = rf_pred >= 0.5 # 0.5 이상인 확률 값은 1로 치환 그렇지 않은 확률은 0으로 치환 
    
# rf_pred = rf_pred.astype(int)
# print(rf_pred) # 이게 제출물이다 이말이야 

0.6314285714285715
0.6657142857142857
0.6371428571428571
0.6628571428571428
0.6114285714285714
0.6028571428571429
0.5971428571428572
0.6
0.6142857142857143
0.6314285714285715


In [43]:
from lightgbm import LGBMClassifier

In [44]:
lgbm_pred = np.zeros((test.shape[0])) # 초기화 리스트 생성 
for tr_idx, val_idx in skf.split(X, y) : # stratifiedKFold로 나눠주기 
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    model = LGBMClassifier(random_state = 42) # 모델 생성
    model.fit(tr_x, tr_y) # 모델 학습하고 
    
    
    val_pred = model.predict_proba(val_x)[:, 1] 
    
    
    ###
    val_pred = val_pred >= 0.5 # 0.5 이상인 확률 값은 1로 치환 그렇지 않은 확률은 0으로 치환
    val_pred = val_pred.astype(int)
    ###
    
    val_score = accuracy_score(val_y, val_pred)
    print(val_score)
    fold_pred = model.predict_proba(test)[:, 1] / 10
    lgbm_pred += fold_pred

0.6428571428571429
0.64
0.6171428571428571
0.6485714285714286
0.6571428571428571
0.6285714285714286
0.5857142857142857
0.6085714285714285
0.6285714285714286
0.6314285714285715


In [49]:
ens = (rf_pred + lgbm_pred) / 2 # 두개를 평균해준다
ens = ens >= 0.5 # 0.5 이상인 확률 값은 1로 치환 그렇지 않은 확률은 0으로 치환 
    
ens = ens.astype(int)

In [48]:
test.head(11)

,총구매액,최대구매액,환불금액,내점일수,내점당구매건수,주말방문비율,구매주기
0,70900400,22000000,4050000.0,13,1.461538,0.789474,26
1,310533100,38558000,48034700.0,90,2.433333,0.369863,3
2,305264140,14825000,30521000.0,101,14.623762,0.083277,3
3,7594080,5225000,0.0,5,2.000000,0.000000,47
4,1795790,1411200,0.0,3,2.666667,0.125000,8
5,13000000,2160000,0.0,5,2.200000,0.000000,61
6,420892520,69920000,39566000.0,144,2.618056,0.249337,2
7,1152000,1152000,0.0,1,1.000000,0.000000,0
8,4825000,4825000,0.0,1,1.000000,0.000000,0
9,110787210,19255600,23062000.0,28,3.321429,0.376344,12


In [50]:
test_pred = pd.DataFrame(ens)

In [51]:
test_pred

,0
0,0
1,0
2,0
3,0
4,0
...,...
2477,1
2478,0
2479,0
2480,0


In [52]:
submission = pd.DataFrame({'custid' : test.cust_id, 'gender' : rf_pred}) # 데이터프레임 만드는 법 알아두자!

AttributeError: ignored

In [47]:
submission['gender'] = ens

NameError: ignored

In [ ]:
submission

,custid,gender
0,3500,0.448789
1,3501,0.162367
2,3502,0.124466
3,3503,0.430447
4,3504,0.381241
...,...,...
2477,5977,0.695288
2478,5978,0.404388
2479,5979,0.415811
2480,5980,0.386653


In [ ]:
submission['gender'] = ens

,custid,gender
0,3500,0.469
1,3501,0.209
2,3502,0.186
3,3503,0.481
4,3504,0.410
...,...,...
2477,5977,0.635
2478,5978,0.171
2479,5979,0.470
2480,5980,0.334


### 주의> 분류 문제에서 앙상블을 할 때는 무조건 predict_proba를 한다.
- predict -> class를 반환
- predict_proba -> probability를 반환

***
- 하드 방식 : 클래스를 예측하고 앙상블 하는 방법으로 다수결 원리
- 소프트 방식 : 확률로 예측하고 앙상블 하는 방법 -> 정확도, precision, recall_score, f1_score, roc_auc_score 다 가능


[0, 1, 1, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
rf_pred

array([0.469, 0.209, 0.186, ..., 0.47 , 0.334, 0.608])

In [ ]:
submission = pd.DataFrame({'custid' : test.cust_id, 'gender' : rf_pred})

In [ ]:
submission

,custid,gender
0,3500,0.469
1,3501,0.209
2,3502,0.186
3,3503,0.481
4,3504,0.410
...,...,...
2477,5977,0.635
2478,5978,0.171
2479,5979,0.470
2480,5980,0.334


In [ ]:
submission.to_csv('sample_submission.csv', index = False) # csv파일로 만들기 알아두자!

In [ ]:
rf_pred = rf_pred / 10

- 1, 2, 3의 평균을 구해라 -> (1 + 2 + 3) / 3 = 2
- 1/3 + 2/3 + 3/3 = 2

***
### <font color = 'red'> Quiz> 앙상블로 Regression 문제 해결! - price 예측 & 평가지표 : MAE

In [ ]:
train = pd.read_csv("house_sales_train.csv")
test = pd.read_csv('house_sales_test.csv')

In [ ]:
train = train.iloc[:, 2:]
test = test.iloc[:, 2:]

### RandomForest + XGBoost + LightGBM

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

In [ ]:
rf_pred = np.zeros((test.shape[0]))

In [ ]:
len(rf_pred), test.shape[0]

(6613, 6613)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

In [ ]:
kf = KFold(n_splits = 10, random_state = 42, shuffle = True)

In [ ]:
X = train.drop('price', axis = 1)
y = train.price

In [ ]:
target = test.drop('price', axis = 1)

In [ ]:
for tr_idx, val_idx in kf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    rf = RandomForestRegressor(random_state = 42)
    
    rf.fit(tr_x, tr_y)
    
    val_pred = rf.predict(val_x)
    val_mae = mean_absolute_error(val_y, val_pred)
    print(val_mae)
    
    fold_pred = rf.predict(target) / 10
    rf_pred += fold_pred

73341.16792414286
64378.02854652958
68901.5862498148
68793.34251000002
77823.88373585715
71763.55813657143
73478.30539715874
67977.45308170898
73384.17159203174
65563.01347101587


### LGBM

In [ ]:
lgbm_pred = np.zeros((target.shape[0]))
mae_list = []

In [ ]:
for tr_idx, val_idx in kf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    lgbm = LGBMRegressor(random_state = 42)
    
    lgbm.fit(tr_x, tr_y)
    
    val_pred = lgbm.predict(val_x)
    val_mae = mean_absolute_error(val_y, val_pred)
    print(val_mae)
    mae_list.append(val_mae)
    
    fold_pred = lgbm.predict(target) / 10
    lgbm_pred += fold_pred

72678.6988325384
63464.38847128298
65716.3684644482
66087.59761285775
72313.0572630867
67760.88870545494
70584.55262242688
65135.567493456016
70372.18188371092
63502.88117387171


In [ ]:
np.mean(mae_list)

68344.08318956819

### XGBoost

In [ ]:
xgb_pred = np.zeros((target.shape[0]))
mae_list = []

In [ ]:
for tr_idx, val_idx in kf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    xgb = XGBRegressor(random_state = 42)
    
    xgb.fit(tr_x, tr_y)
    
    val_pred = xgb.predict(val_x)
    val_mae = mean_absolute_error(val_y, val_pred)
    print(val_mae)
    mae_list.append(val_mae)
    
    fold_pred = xgb.predict(target) / 10
    xgb_pred += fold_pred

70546.39083854167
63967.39294791667
67513.69255208333
68129.62514583333
75464.90358333333
71663.354578125
71124.68770833334
65147.80371875
70971.4149453125
64736.21525


In [ ]:
np.mean(mae_list)

68926.54812682292

In [ ]:
final_pred = (rf_pred + lgbm_pred + xgb_pred) / 3

In [ ]:
test['prediction'] = final_pred

In [ ]:
mean_absolute_error(test['price'], test['prediction'])

64092.57481471857